In [8]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm.session import Session, sessionmaker
from sqlalchemy import create_engine
from sqlalchemy.schema import MetaData, Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.orm import relationship

In [ ]:
base.metadata.clear(), engine.dispose()

In [208]:
base.metadata.clear(), engine.dispose()
engine = create_engine('sqlite:///C:/Users/KimYM/Documents/Medicine/medicine_db.db')
meta = MetaData(engine)
base = declarative_base()

In [370]:
base.metadata.clear(), engine.dispose()
engine = create_engine('sqlite:///:memory:')
meta = MetaData(engine)
base = declarative_base()

In [371]:
class Medicine(base):
    __tablename__ = 'T_Medicine'
    
    name = Column('name', Text, primary_key=True, autoincrement=False)
    content_url = Column('content_url', Text)
    identifier_front = Column('identifier_front', Text)
    identifier_back = Column('identifier_back', Text)
    color_front = Column('color_front', Text)
    color_back = Column('color_back', Text)
    shape = Column('shape', Text)
    form_code = Column('form_code', Text)
    image_url = Column('image_url', Text)
    logo_num = Column('logo_num', ForeignKey('T_Logo.logo_num'), nullable=True)
    company_name = Column('company_name', ForeignKey('T_Company.name'), nullable=False)
    
    company = relationship('Company', back_populates='medicines')
    logo = relationship('Logo', back_populates='medicines')
    
    def __init__(self, name, company, content_url=None, iden_front=None, iden_back=None, form_code=None,
                 image_url=None, color=None, shape=None,logo_num=None):
        self.name = name
        self.content_url = content_url
        self.identifier_front = iden_front
        self.identifier_back = iden_back
        self.color = color
        self.shape = shape
        self.form_code = form_code
        self.image_url = image_url
        self.logo_num = logo_num
        self.company_name = company

        session.add(self)
        session.commit()
        
        if logo_num is not None:
            if session.query(Logo.logo_num).filter(Logo.logo_num == logo_num).first() is None:
                print(self.name,':There is no logo image')
                self.logo_num = None
        if session.query(Company.name).filter(Company.name == company).first() is None:
            Company(company)
        
    def info(self):
        print('제조사:', self.company_name)
        print('품목명:', self.name)
        print('식별문자앞:', self.identifier_front)
        print('식별문자뒤:', self.identifier_back)
        print('색상:', self.color)
        print('모양:', self.shape)
        print('제형코드이름:', self.form_code)
        print('로고번호:', self.logo_num)
        print('상세페이지:', self.content_url)
        print()
        
    
class Company(base):
    __tablename__ = 'T_Company'
    
    name = Column('name', Text, primary_key=True, autoincrement=False)
    
    medicines = relationship('Medicine', back_populates='company', uselist=True)
    
    def __init__(self, name):
        self.name = name
        session.add(self)
        session.commit()

    def get_medicines(self):
        medi = list()
        print('제조사:', self.name)
        for medicine in self.medicines:
            print('    제품명:', medicine.name)
        print()
            
class Logo(base):
    __tablename__ = 'T_Logo'
    
    logo_num = Column('logo_num', Text, primary_key=True, autoincrement=False)
    image_path = Column('image_path', Text, nullable=False)
    
    medicines = relationship('Medicine', back_populates='logo', uselist=True)
    
    def __init__(self, logo_num, path):
        self.logo_num = logo_num
        self.image_path = path
        session.add(self)
        session.commit()
        
    def get_medicines(self):
        medi = list()
        print('로고번호:', self.logo_num)
        print('이 로고를 사용하는 제품들')
        for medicine in self.medicines:
            print('    제품명:', medicine.name)
        print('이미지:', self.image_path)
        print()

In [372]:
Session = sessionmaker(engine)
session = Session()
base.metadata.create_all(engine)

In [373]:
Logo(1, 'aaa')
Logo(2, 'bbb')
Logo(3, 'ccc')

In [374]:
Medicine('name1', 'company2', logo_num=2)
Medicine('name2', 'company1', logo_num=4)
Medicine('name3', 'company3', logo_num=1)
Medicine('name4', 'company4', logo_num=3)
Medicine('name5', 'company1', logo_num=2)
Medicine('name6', 'company3', logo_num=1)
Medicine('name7', 'company2', logo_num=2)
Medicine('name8', 'company1')

name2 :There is no logo image


In [375]:
for _ in session.query(Company).all():
    _.get_medicines()

제조사: company2
    제품명: name1
    제품명: name7

제조사: company1
    제품명: name2
    제품명: name5
    제품명: name8

제조사: company3
    제품명: name3
    제품명: name6

제조사: company4
    제품명: name4



In [376]:
for _ in session.query(Logo).all():
    _.get_medicines()

로고번호: 1
이 로고를 사용하는 제품들
    제품명: name3
    제품명: name6
이미지: aaa

로고번호: 2
이 로고를 사용하는 제품들
    제품명: name1
    제품명: name5
    제품명: name7
이미지: bbb

로고번호: 3
이 로고를 사용하는 제품들
    제품명: name4
이미지: ccc



In [4]:
import pandas as pd

In [14]:
medicine = pd.read_csv('C:/Users/KimYM/Documents/카카오톡 받은 파일/merged_med.csv')

In [15]:
medicine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18776 entries, 0 to 18775
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   복사품목명           18776 non-null  object
 1   품목일련번호          18776 non-null  int64 
 2   품목명             18776 non-null  object
 3   업소일련번호          18776 non-null  int64 
 4   업소명             18776 non-null  object
 5   성상              18776 non-null  object
 6   큰제품이미지          18776 non-null  object
 7   표시앞             18775 non-null  object
 8   표시뒤             18774 non-null  object
 9   의약품제형           18776 non-null  object
 10  색상앞             18776 non-null  object
 11  색상뒤             18776 non-null  object
 12  분할선앞            18776 non-null  object
 13  분할선뒤            18776 non-null  object
 14  크기장축            18776 non-null  object
 15  크기단축            18776 non-null  object
 16  크기두께            18776 non-null  object
 17  이미지생성일자(약학정보원)  18776 non-null  int64 
 18  분류번호  

In [17]:
medicine[medicine.표시뒤.isna()]

,복사품목명,품목일련번호,품목명,업소일련번호,업소명,성상,큰제품이미지,표시앞,표시뒤,의약품제형,...,품목허가일자,제형코드명,표기내용앞,표기내용뒤,표기이미지앞,표기이미지뒤,표기코드앞,표기코드뒤,변경일자,url
8343,센나르정,200101225,센나르정(나프록센나트륨),19550015,(주)동구바이오제약,엷은청색의원형필름코팅정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,마크,NaN,원형,...,20010419,필름코팅정,D,-,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,-,r0026,-,-,http://medinavi.co.kr/pill/p/200101225.html
15266,리코나졸정,201603868,리코나졸정(플루코나졸),20040011,(주)씨티씨바이오,분홍색의장방형정제,https://nedrug.mfds.go.kr/pbp/cmn/itemImageDow...,150,NaN,타원형,...,20160624,나정,-,-,-,-,-,-,20210625,http://medinavi.co.kr/pill/p/201603868.html


In [19]:
medicine.표시뒤.value_counts()

-            9443
분할선          1457
마크            481
10            256
5             210
             ... 
S분할선V           1
DKP             1
RZC             1
ΛSF분할선200       1
ABP             1
Name: 표시뒤, Length: 2480, dtype: int64